In [1]:
import sys
sys.path.insert(0, '../Lab/')
sys.path.insert(0, '../New_Hybrids/')
from KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from KNN.UserKNNCFRecommender import UserKNNCFRecommender
from UserIcmKNNCFRecommender import UserIcmKNNCFRecommender
from ItemIcmKNNCFRecommender import ItemIcmKNNCFRecommender
from WeightedHybridV2 import WeightedHybridScoreRecommender
from optimize_weights import optimize_weights
from SLIM_BPR.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from GraphBased.P3alphaRecommender import P3alphaRecommender
from GraphBased.RP3betaRecommender import RP3betaRecommender
from MatrixFactorization.IALSRecommender import IALSRecommender
from Base.Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
import matplotlib
import matplotlib.pyplot as plt
from Base.Similarity.Compute_Similarity_Python import Compute_Similarity_Python

%matplotlib inline 

import numpy as np
sys.path.insert(0, '../datasets/')
from load_tables import load_ICM_csr, load_URM_csr, load_clustered_ICM_csr
def plotta(t,maps):
    fig, ax = plt.subplots()
    ax.plot(t, maps)

    ax.set(xlabel='T', ylabel='MAP')
    ax.grid()
    plt.show()

In [2]:

from ParameterTuning.SearchBayesianSkoptCross import SearchBayesianSkoptCross
from ParameterTuning.SearchAbstractClassCross import SearchInputRecommenderArgs
from skopt.space import Real, Integer, Categorical

In [3]:
#ICM_all = load_ICM_csr()
#ICM_cl = load_clustered_ICM_csr()
URM_all = load_URM_csr()

In [4]:

#URM_train, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.9)
#URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train, train_percentage = 0.8)

#evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
#evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[ 10])

In [8]:
from ParameterTuning.SearchAbstractClass import SearchInputRecommenderArgs


def hyperparameter_optimization_CFW_D(URM_train, ICM_train, W_sparse_CF):

    recommender_class = UserKNNCFRecommender

    parameterSearch = SearchBayesianSkoptCross(recommender_class,
                                               n_validations = 2,
                                     evaluator_validation=None,
                                     evaluator_test=None)


    hyperparameters_range_dictionary = {}
    hyperparameters_range_dictionary["topK"] = Integer(5, 1000)
    hyperparameters_range_dictionary["shrink"] = Integer(0, 100)

    #hyperparameters_range_dictionary["add_zeros_quota"] = Real(low = 0, high = 1, prior = 'uniform')
    #hyperparameters_range_dictionary["normalize_similarity"] = Categorical([True, False])


    recommender_input_args = SearchInputRecommenderArgs(
        CONSTRUCTOR_POSITIONAL_ARGS = [URM_train],
        CONSTRUCTOR_KEYWORD_ARGS = {},
        FIT_POSITIONAL_ARGS = [],
        FIT_KEYWORD_ARGS = {}
    )


    output_folder_path = "result_experiments/"

    import os

    # If directory does not exist, create
    if not os.path.exists(output_folder_path):
        os.makedirs(output_folder_path)

    n_cases = 1000
    metric_to_optimize = "MAP"


    # Clone data structure to perform the fitting with the best hyperparameters on train + validation data
    recommender_input_args_last_test = recommender_input_args.copy()
    recommender_input_args_last_test.CONSTRUCTOR_POSITIONAL_ARGS[0] = URM_train


    parameterSearch.search(recommender_input_args,
                           recommender_input_args_last_test = recommender_input_args_last_test,
                           parameter_search_space = hyperparameters_range_dictionary,
                           n_cases = n_cases,
                           n_random_starts = int(n_cases/3),
                           save_model = "no",
                           output_folder_path = output_folder_path,
                           output_file_name_root = recommender_class.RECOMMENDER_NAME,
                           metric_to_optimize = metric_to_optimize
                          )

In [ ]:
hyperparameter_optimization_CFW_D(URM_all, None, None)

Iteration No: 1 started. Evaluating function at random point.
UserKNNCFRecommender: URM Detected 66 (0.83 %) cold users.
UserKNNCFRecommender: URM Detected 2376 (9.15 %) cold items.
SearchBayesianSkoptCross: Testing config: {'topK': 48, 'shrink': 3}
Similarity column 7947 ( 100 % ), 3557.11 column/sec, elapsed time 0.04 min
EvaluatorHoldout: Processed 5657 ( 100.00% ) in 10.44 sec. Users per second: 542
UserKNNCFRecommender: URM Detected 76 (0.96 %) cold users.
UserKNNCFRecommender: URM Detected 2384 (9.18 %) cold items.
SearchBayesianSkoptCross: Testing config: {'topK': 48, 'shrink': 3}
Similarity column 7947 ( 100 % ), 4625.88 column/sec, elapsed time 0.03 min
EvaluatorHoldout: Processed 5708 ( 100.00% ) in 10.01 sec. Users per second: 570
UserKNNCFRecommender: URM Detected 76 (0.96 %) cold users.
UserKNNCFRecommender: URM Detected 2384 (9.18 %) cold items.
SearchBayesianSkoptCross: Testing config: {'topK': 48, 'shrink': 3}
Similarity column 7947 ( 100 % ), 4865.85 column/sec, elapse

EvaluatorHoldout: Processed 5657 ( 100.00% ) in 9.31 sec. Users per second: 608
UserKNNCFRecommender: URM Detected 76 (0.96 %) cold users.
UserKNNCFRecommender: URM Detected 2384 (9.18 %) cold items.
SearchBayesianSkoptCross: Testing config: {'topK': 550, 'shrink': 94}
Similarity column 7947 ( 100 % ), 5233.58 column/sec, elapsed time 0.03 min
EvaluatorHoldout: Processed 5708 ( 100.00% ) in 9.17 sec. Users per second: 623
UserKNNCFRecommender: URM Detected 76 (0.96 %) cold users.
UserKNNCFRecommender: URM Detected 2384 (9.18 %) cold items.
SearchBayesianSkoptCross: Testing config: {'topK': 550, 'shrink': 94}
Similarity column 7947 ( 100 % ), 6150.68 column/sec, elapsed time 0.02 min
SearchBayesianSkoptCross: Config 6 is suboptimal. Config: {'topK': 550, 'shrink': 94} - results: FKN RESULT 0.05007126235417701

Iteration No: 7 ended. Evaluation done at random point.
Time taken: 23.2327
Function value obtained: 0.0501
Current minimum: 0.0501
Iteration No: 8 started. Evaluating function at

Similarity column 7947 ( 100 % ), 4879.96 column/sec, elapsed time 0.03 min
SearchBayesianSkoptCross: Config 12 is suboptimal. Config: {'topK': 675, 'shrink': 48} - results: FKN RESULT 0.049500928115387366

Iteration No: 13 ended. Evaluation done at random point.
Time taken: 24.0102
Function value obtained: 0.0495
Current minimum: 0.0495
Iteration No: 14 started. Evaluating function at random point.
UserKNNCFRecommender: URM Detected 66 (0.83 %) cold users.
UserKNNCFRecommender: URM Detected 2376 (9.15 %) cold items.
SearchBayesianSkoptCross: Testing config: {'topK': 18, 'shrink': 4}
Similarity column 7947 ( 100 % ), 5828.78 column/sec, elapsed time 0.02 min
EvaluatorHoldout: Processed 5657 ( 100.00% ) in 8.12 sec. Users per second: 696
UserKNNCFRecommender: URM Detected 76 (0.96 %) cold users.
UserKNNCFRecommender: URM Detected 2384 (9.18 %) cold items.
SearchBayesianSkoptCross: Testing config: {'topK': 18, 'shrink': 4}
Similarity column 7947 ( 100 % ), 5856.42 column/sec, elapsed tim

Similarity column 7947 ( 100 % ), 3110.09 column/sec, elapsed time 0.04 min
EvaluatorHoldout: Processed 5657 ( 100.00% ) in 17.71 sec. Users per second: 319
UserKNNCFRecommender: URM Detected 76 (0.96 %) cold users.
UserKNNCFRecommender: URM Detected 2384 (9.18 %) cold items.
SearchBayesianSkoptCross: Testing config: {'topK': 852, 'shrink': 9}
Similarity column 7947 ( 100 % ), 2445.06 column/sec, elapsed time 0.05 min
EvaluatorHoldout: Processed 5708 ( 100.00% ) in 18.43 sec. Users per second: 310
UserKNNCFRecommender: URM Detected 76 (0.96 %) cold users.
UserKNNCFRecommender: URM Detected 2384 (9.18 %) cold items.
SearchBayesianSkoptCross: Testing config: {'topK': 852, 'shrink': 9}
Similarity column 7947 ( 100 % ), 6472.72 column/sec, elapsed time 0.02 min
SearchBayesianSkoptCross: Config 19 is suboptimal. Config: {'topK': 852, 'shrink': 9} - results: FKN RESULT 0.05109526001155505

Iteration No: 20 ended. Evaluation done at random point.
Time taken: 44.7880
Function value obtained: 0

UserKNNCFRecommender: URM Detected 76 (0.96 %) cold users.
UserKNNCFRecommender: URM Detected 2384 (9.18 %) cold items.
SearchBayesianSkoptCross: Testing config: {'topK': 123, 'shrink': 75}
Similarity column 7947 ( 100 % ), 4795.51 column/sec, elapsed time 0.03 min
SearchBayesianSkoptCross: Config 25 is suboptimal. Config: {'topK': 123, 'shrink': 75} - results: FKN RESULT 0.05242625815204227

Iteration No: 26 ended. Evaluation done at random point.
Time taken: 27.2453
Function value obtained: 0.0524
Current minimum: 0.0444
Iteration No: 27 started. Evaluating function at random point.
UserKNNCFRecommender: URM Detected 66 (0.83 %) cold users.
UserKNNCFRecommender: URM Detected 2376 (9.15 %) cold items.
SearchBayesianSkoptCross: Testing config: {'topK': 832, 'shrink': 59}
Similarity column 7947 ( 100 % ), 5270.40 column/sec, elapsed time 0.03 min
EvaluatorHoldout: Processed 5657 ( 100.00% ) in 15.22 sec. Users per second: 372
UserKNNCFRecommender: URM Detected 76 (0.96 %) cold users.
Us

...

Similarity column 7947 ( 100 % ), 7048.25 column/sec, elapsed time 0.02 min
EvaluatorHoldout: Processed 5657 ( 100.00% ) in 7.48 sec. Users per second: 756
UserKNNCFRecommender: URM Detected 76 (0.96 %) cold users.
UserKNNCFRecommender: URM Detected 2384 (9.18 %) cold items.
SearchBayesianSkoptCross: Testing config: {'topK': 1000, 'shrink': 100}
Similarity column 7947 ( 100 % ), 6949.25 column/sec, elapsed time 0.02 min
EvaluatorHoldout: Processed 5708 ( 100.00% ) in 6.55 sec. Users per second: 872
UserKNNCFRecommender: URM Detected 76 (0.96 %) cold users.
UserKNNCFRecommender: URM Detected 2384 (9.18 %) cold items.
SearchBayesianSkoptCross: Testing config: {'topK': 1000, 'shrink': 100}
Similarity column 7947 ( 100 % ), 8700.76 column/sec, elapsed time 0.02 min
SearchBayesianSkoptCross: Config 383 is suboptimal. Config: {'topK': 1000, 'shrink': 100} - results: FKN RESULT 0.04838280873637735

Iteration No: 384 ended. Search finished for the next optimal point.
Time taken: 33.0189
Functi

/Users/08volt/opt/anaconda3/lib/python3.7/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Similarity column 7947 ( 100 % ), 7367.36 column/sec, elapsed time 0.02 min
EvaluatorHoldout: Processed 5657 ( 100.00% ) in 6.37 sec. Users per second: 888
UserKNNCFRecommender: URM Detected 76 (0.96 %) cold users.
UserKNNCFRecommender: URM Detected 2384 (9.18 %) cold items.
SearchBayesianSkoptCross: Testing config: {'topK': 1000, 'shrink': 100}
Similarity column 7947 ( 100 % ), 8868.71 column/sec, elapsed time 0.01 min
EvaluatorHoldout: Processed 5708 ( 100.00% ) in 5.92 sec. Users per second: 964
UserKNNCFRecommender: URM Detected 76 (0.96 %) cold users.
UserKNNCFRecommender: URM Detected 2384 (9.18 %) cold items.
SearchBayesianSkoptCross: Testing config: {'topK': 1000, 'shrink': 100}
Similarity column 7947 ( 100 % ), 7297.68 column/sec, elapsed time 0.02 min
SearchBayesianSkoptCross: Config 384 is suboptimal. Config: {'topK': 1000, 'shrink': 100} - results: FKN RESULT 0.04838280873637735

Iteration No: 385 ended. Search finished for the next optimal point.
Time taken: 30.9007
Functi

/Users/08volt/opt/anaconda3/lib/python3.7/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Similarity column 7947 ( 100 % ), 8688.89 column/sec, elapsed time 0.02 min
EvaluatorHoldout: Processed 5657 ( 100.00% ) in 7.63 sec. Users per second: 742
UserKNNCFRecommender: URM Detected 76 (0.96 %) cold users.
UserKNNCFRecommender: URM Detected 2384 (9.18 %) cold items.
SearchBayesianSkoptCross: Testing config: {'topK': 1000, 'shrink': 100}


In [7]:
#Config 14: {'topK': 52, 'add_zeros_quota': 0.8588004964327149, 'normalize_similarity': False} itemknn